### Authentication Wrapper

- In this example, the `authenticate` decorator function takes in a function `func` as its parameter. 

- It returns a `wrapper_authenticate` function that checks if the instance variable `is_authenticated` of the object that func is being called on is True. 

    - If it is not True, the function prints an error message and returns None. 

    - If is_authenticated is True, the wrapper function calls func with the object that it is being called on and its arguments and returns its output.

In [1]:
import functools

def authenticate(func):
    """
    A decorator that requires authentication for a class method.

    This decorator takes a class method as an argument and returns a wrapped function that requires 
    authentication to be performed before the method is executed. The wrapped function checks whether 
    the class instance has the `is_authenticated` attribute set to True. If it is, the wrapped function 
    calls the original method with the given arguments. If not, it prints an error message and returns None.

    The decorator uses the functools.wraps() function to ensure that the wrapped function has the same
    name, documentation, and other attributes as the original method. 

    Args:
        func: The class method to be wrapped with authentication functionality.

    Returns:
        The wrapped function that requires authentication to be performed before the original method 
        is executed. If authentication fails, it prints an error message and returns None. Otherwise, 
        it calls the original method with the given arguments.
    """
    @functools.wraps(func)
    def wrapper_authenticate(self, *args, **kwargs):
        if not self.is_authenticated:
            print("******************************************************")
            print('You must be authenticated to perform this transaction!')
            print("******************************************************")
            return None 
        else:
            return func(self, *args, **kwargs)
    
    return wrapper_authenticate


- The `BankAccount` class is a class that simulates a bank account. 

- It has instance variables `account_number` and `balance`, and a class variable `accounts` that is a list of valid account numbers. 

- It also has methods `deposit`, `withdraw`, and `transfer` that each take in an amount and modify the balance of the account. 

- Each of these methods is decorated with the authenticate_wrapper decorator function, which ensures that the method can only be called if the `is_authenticated` instance variable of the object that the method is being called on is True. 

- The BankAccount class also has a method `authenticate` that takes in a password and sets the is_authenticated instance variable to True if the password is correct.

- The `__str__` method of the BankAccount class returns a string representation of the account that includes its account number and balance.

- The code:

    - creates two instances of the BankAccount class, `account1` and `account2`, and calls the authenticate, deposit, withdraw, and transfer methods on account1. 

    - prints the string representations of account1 and account2.

In [2]:
class BankAccount:
    """
    A class representing a bank account.

    This class provides methods for depositing, withdrawing, and transferring funds between accounts, 
    as well as an authentication method to ensure that only authorized users can perform transactions. 

    Attributes:
        accounts (list): A list of valid account numbers.
        account_number (str): The account number associated with the account.
        balance (float): The current balance of the account.
        is_authenticated (bool): A flag indicating whether the user is authenticated.

    Methods:
        __init__(self, account_number, balance): Initializes a new BankAccount instance with the given 
            account number and balance.
        deposit(self, amount): Deposits the given amount into the account.
        withdraw(self, amount): Withdraws the given amount from the account.
        transfer(self, amount, recipient): Transfers the given amount from the account to the recipient's 
            account.
        __str__(self): Returns a string representation of the BankAccount instance.
        authenticate(self, password): Authenticates the user with the given password.

    """
    
    accounts = ['000100', '000101', '000200', '000201'] # list of valid accounts
    
    def __init__(self, account_number, balance):
        self.account_number = account_number
        self.balance = balance 
        self.is_authenticated = False 
        
    @authenticate
    def deposit(self, amount):
        self.balance += amount 
        print(f"Deposited {amount}. New balance: {self.balance}.")
    
    @authenticate
    def withdraw(self, amount):
        if self.balance >= amount:
            self.balance -= amount 
            print(f"Withdrew ${amount}. New balance: ${self.balance}.")
        else:
            print("Insufficient funds!")
    
    @authenticate
    def transfer(self, amount, recipient):
        if self.balance >= amount and recipient.account_number in BankAccount.accounts:
            self.withdraw(amount)
            recipient.balance += amount
            print(f"Transferred ${amount} to account {recipient.account_number}.")
        else: 
            print('Insufficient funds!')
            
    def __str__(self):
        return f"Account {self.account_number}: ${self.balance}"
    
    def authenticate(self, password):
        if password in ['secret', 'password']:
            self.is_authenticated = True 
            print("Authentication successful. You are in!")
        else:
            print("Authetication failed.")
            

account1 = BankAccount('000100', 1_000)
account2 = BankAccount('000101', 8_000)

account1.authenticate('secret')
account1.deposit(2_000)
account1.withdraw(200)
account1.transfer(1_000, account2)

print(account1)
print(account2)


Authentication successful. You are in!
Deposited 2000. New balance: 3000.
Withdrew $200. New balance: $2800.
Withdrew $1000. New balance: $1800.
Transferred $1000 to account 000101.
Account 000100: $1800
Account 000101: $9000


### `When authentication fails...`

In [3]:
account1 = BankAccount('000100', 1_000)
account2 = BankAccount('000101', 8_000)

account1.authenticate('wrong secret')
account1.deposit(2_000)
account1.withdraw(200)
account1.transfer(1_000, account2)

print(account1)
print(account2)

Authetication failed.
******************************************************
You must be authenticated to perform this transaction!
******************************************************
******************************************************
You must be authenticated to perform this transaction!
******************************************************
******************************************************
You must be authenticated to perform this transaction!
******************************************************
Account 000100: $1000
Account 000101: $8000
